In [1]:
import os
import pandas as pd
from datetime import timedelta
from tqdm import tqdm


def load_processed_data_for_day(file_path):
    """
    加载处理后的SMART数据文件（每个日期的文件）
    """
    return pd.read_csv(file_path)

def load_all_smart_data(smart_data_base_folder):
    """
    一次性加载所有SMART数据文件到内存。
    """
    all_smart_data = []

    # 遍历文件夹中所有csv文件
    for filename in tqdm(os.listdir(smart_data_base_folder)[:30]):
        if filename.endswith('_processed.csv'):
            file_path = os.path.join(smart_data_base_folder, filename)
            smart_data = pd.read_csv(file_path)
            smart_data['ds'] = pd.to_datetime(smart_data['ds'])  # 确保 'ds' 列为 datetime 类型
            all_smart_data.append(smart_data)

    # 合并所有的 DataFrame
    return pd.concat(all_smart_data, ignore_index=True)


def find_files_in_date_range(base_folder, start_date, end_date):
    """
    根据给定的日期范围查找需要加载的文件
    """
    # 将日期范围转为字符串（例如：20180101）
    start_date_str = start_date.strftime('%Y%m%d')
    end_date_str = end_date.strftime('%Y%m%d')
    
    # 获取所有文件并按日期排序
    all_files = sorted([f for f in os.listdir(base_folder) if f.endswith('_processed.csv')])
    
    # 根据日期范围筛选文件
    relevant_files = [os.path.join(base_folder, f) for f in all_files if start_date_str <= f[:8] <= end_date_str]
    
    return relevant_files



def generate_features_and_labels(failure_data, smart_data_base_folder, lookback_days, window_size, step_size):
    """
    生成模型训练数据，按给定的滑动窗口选择特征和标签。
    """
    all_data = []

    # 按照 'failure_time' 升序排序 failure_data
    failure_data = failure_data.sort_values(by='failure_time')
    
    print(f'Loading Data ...')
    # 预先加载所有的SMART数据
    all_smart_data = load_all_smart_data(smart_data_base_folder)
    print(f'Loading Data Success')

    # 过滤掉在给定时间范围内的失败数据
    failure_data = failure_data[(failure_data['failure_time'] >= start_date) & (failure_data['failure_time'] <= end_date)]

    # 生成特征和标签
    for failure in tqdm(failure_data.itertuples(), desc="Processing failure events"):
        failure_time = failure.failure_time
        disk_id = failure.disk_id
        model = failure.model
        
        # 计算前 lookback_days 天的时间范围
        start_time = failure_time - timedelta(days=lookback_days)
        end_time = failure_time
        
        # 筛选出在时间范围内的SMART数据
        relevant_smart_data = all_smart_data[(all_smart_data['disk_id'] == disk_id) & 
                                             (all_smart_data['model'] == model) & 
                                             (all_smart_data['ds'] >= start_time) & 
                                             (all_smart_data['ds'] <= end_time)]

        # 滑动窗口处理
        for i in range(0, lookback_days - window_size + 1, step_size):
            window_data = []

            # 获取窗口内的时间段
            window_start_time = failure_time - timedelta(days=(lookback_days - i))
            window_end_time = window_start_time + timedelta(days=window_size)
            
            # 筛选出时间段内的数据
            window_data_for_window = relevant_smart_data[(relevant_smart_data['ds'] >= window_start_time) & 
                                                        (relevant_smart_data['ds'] < window_end_time)]
            
            if not window_data_for_window.empty:
                # 获取该窗口内的所有特征
                values = window_data_for_window.drop(columns=['disk_id', 'label']).values.tolist()

                # 获取窗口内最后一条数据
                last_window_data = window_data_for_window.iloc[-1]
                
                # 计算标签，基于最后一条数据的时间
                label = last_window_data['label']
                
                # 将窗口内的特征和标签添加到 all_data 列表
                all_data.append({'data': values, 'label': label})
                print(all_data[-1])
    
    return all_data



In [53]:
import torch
from torch.utils.data import Dataset

In [ ]:
smart_data_base_folder = '/mnt/raid5/sum/card/storage/StreamDFP/dataset/processed_smart_data'  # 设置为存放处理过的数据文件夹


lookback_days = 20  # 前30天的数据
window_size = 10  # 每个窗口大小为10天
step_size = 5  # 每次滑动1天
separator = '<SEP>'

In [ ]:
class SlidingWindowDataset(Dataset):
    def __init__(self, smart_data_base_folder, lookback_days=None, window_size = 10, step_size = 5, separator='<SEP>'):
        """
        初始化 Dataset 类
        :param data: pandas.DataFrame，包含滑动窗口后的数据，每行是一个样本
        :param tokenizer: 可选，用于将文本序列转换为 token 的函数
        :param separator: 分隔符，用于分割窗口内的数据
        """
        self.smart_data_base_folder = smart_data_base_folder
        self.lookback_days = lookback_days
        self.window_size = window_size
        self.step_size = step_size
        self.separator = separator

    def __len__(self):
        """
        返回数据集大小
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        获取数据集中的第 idx 个样本
        :param idx: 索引
        :return: 样本字典，包括序列化数据和标签
        """
        # 获取序列化数据和标签
        row = self.data.iloc[idx]
        serialized_data = row['serialized_data']
        label = row['label']
        
        # 可选：将序列化数据 token 化（如果提供了 tokenizer）
        if self.tokenizer:
            serialized_data = self.tokenizer(self.separator.join(serialized_data))
        else:
            # 默认转换为 tensor
            serialized_data = torch.tensor(serialized_data, dtype=torch.float32)
        
        # 返回样本
        return {
            'data': serialized_data,
            'label': torch.tensor(label, dtype=torch.float32)
        }

In [3]:

# 调用函数的示例
smart_data_base_folder = '/mnt/raid5/sum/card/storage/StreamDFP/dataset/processed_smart_data'  # 设置为存放处理过的数据文件夹


lookback_days = 20  # 前30天的数据
window_size = 10  # 每个窗口大小为10天
step_size = 5  # 每次滑动1天

# 生成特征和标签数据集
dataset = generate_features_and_labels(failure_data, smart_data_base_folder, lookback_days, window_size, step_size)

# 查看生成的DataFrame
print(dataset[0])

Loading Data ...


  3%|▎         | 1/30 [00:00<00:05,  5.65it/s]

100%|██████████| 30/30 [00:11<00:00,  2.58it/s]


Loading Data Success


NameError: name 'start_date' is not defined

In [4]:
failure_file_path = '/mnt/raid5/sum/card/storage/StreamDFP/dataset/ssd_failure_label.csv'  # 设置为failure数据的路径

# 加载失败数据
failure_data = pd.read_csv(failure_file_path)
failure_data['failure_time'] = pd.to_datetime(failure_data['failure_time'])

In [5]:
failure_data.head()

,model,failure_time,disk_id
0,MA2,2019-03-22 10:24:38,4711
1,MA2,2019-02-22 04:56:06,82064
2,MA2,2018-12-17 12:16:33,32311
3,MA2,2018-05-19 17:32:03,18316
4,MA2,2018-10-25 04:00:50,32466


首先对failure_data升序排序

In [7]:
failure_data = failure_data.sort_values(by='failure_time', ascending=True)
failure_data

,model,failure_time,disk_id
309,MA2,2018-01-02 19:15:32,33722
293,MA2,2018-01-02 22:45:16,58337
2197,MA1,2018-01-03 03:23:44,26378
2001,MA1,2018-01-03 03:29:27,39876
13424,MC1,2018-01-03 05:03:03,199348
...,...,...,...
16226,MC2,2019-12-31 16:16:05,12463
16066,MC2,2019-12-31 19:28:04,10186
13278,MC1,2019-12-31 19:32:50,18144
602,MA2,2019-12-31 21:57:36,99516


In [38]:
# 因为如果从20180101的错误开始计算，由于没有2018年之前的数据，所以这些输入会不匹配。因此我们直接从2018年1月10日的数据开始计算。
filter_failure_data = pd.to_datetime('2018-01-22')
failure_data = failure_data[failure_data['failure_time'] >= filter_failure_data]
failure_data

,model,failure_time,disk_id
1747,MA1,2018-01-22 03:00:26,20195
1603,MA1,2018-01-22 03:19:40,22822
6902,MC1,2018-01-22 18:31:07,9578
1540,MA1,2018-01-23 03:19:28,15382
508,MA2,2018-01-23 03:19:46,3417
...,...,...,...
16226,MC2,2019-12-31 16:16:05,12463
16066,MC2,2019-12-31 19:28:04,10186
13278,MC1,2019-12-31 19:32:50,18144
602,MA2,2019-12-31 21:57:36,99516


In [37]:
cutoff_date = pd.to_datetime('2018-01-22')

filtered_data = failure_data[failure_data['failure_time'] < cutoff_date]
filtered_data.describe()

,failure_time,disk_id
count,4,4.000000
mean,2018-01-20 09:59:49.750000128,101062.750000
min,2018-01-20 03:20:08,27780.000000
25%,2018-01-20 03:22:58.249999872,57552.000000
50%,2018-01-20 04:44:47,93164.000000
75%,2018-01-20 11:21:38.500000,136674.750000
max,2018-01-21 03:09:37,190143.000000
std,NaN,70119.373402


In [45]:
# 获取failure_data第一条数据的failure_time字段，但是要判断最小的时间是否小于20180101
from tqdm import tqdm
smart_data_base_folder = '/mnt/raid5/sum/card/storage/StreamDFP/dataset/processed_smart_data'  # 设置为存放处理过的数据文件夹

end_date = failure_data.iloc[0].failure_time
start_date = max(end_date - timedelta(days=lookback_days - 1), pd.to_datetime('2018-01-01'))
print(start_date, end_date)
# 获取日期范围
date_range = pd.date_range(start=start_date, end=end_date)

# 读取并合并 CSV 文件
all_data = []

for date in tqdm(date_range):
    # 格式化日期为 YYYYMMDD
    date_str = date.strftime('%Y%m%d')
    file_name = f"{date_str}_processed.csv"
    
    try:
        # 读取 CSV 文件，并附加到 all_data 列表
        df = pd.read_csv(os.path.join(smart_data_base_folder, file_name))
        df['date'] = date_str  # 添加文件日期作为一列
        all_data.append(df)
    except FileNotFoundError:
        print(f"文件 {file_name} 未找到，跳过该文件")

2018-01-03 03:00:26 2018-01-22 03:00:26


100%|██████████| 20/20 [00:07<00:00,  2.82it/s]


In [46]:
range_csv = pd.concat(all_data, ignore_index=True)

In [47]:
range_csv

,disk_id,ds,model,r_1,r_9,r_12,r_171,r_173,r_174,r_183,r_187,r_188,r_194,r_195,r_198,label,date
0,0,2018-01-03,MA1,45406.0,27881.0,32.0,0.0,0.0,30.0,NaN,0.0,0.0,15.0,820719.0,0.0,0,20180103
1,100001,2018-01-03,MA2,NaN,24508.0,19.0,0.0,NaN,17.0,0.0,0.0,NaN,22.0,NaN,NaN,0,20180103
2,100007,2018-01-03,MA2,NaN,32542.0,24.0,0.0,NaN,22.0,0.0,0.0,NaN,23.0,NaN,NaN,0,20180103
3,100016,2018-01-03,MC1,0.0,6974.0,26.0,0.0,45.0,25.0,0.0,0.0,19.0,24.0,0.0,0.0,0,20180103
4,100024,2018-01-03,MC1,1.0,4626.0,23.0,0.0,3.0,14.0,0.0,0.0,16.0,18.0,0.0,0.0,0,20180103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6952088,99902,2018-01-22,MA2,NaN,20697.0,21.0,0.0,NaN,17.0,0.0,0.0,NaN,26.0,NaN,NaN,0,20180122
6952089,99928,2018-01-22,MA2,NaN,20692.0,17.0,0.0,NaN,11.0,0.0,0.0,NaN,26.0,NaN,NaN,0,20180122
6952090,99958,2018-01-22,MC1,0.0,4503.0,21.0,0.0,34.0,20.0,0.0,0.0,0.0,19.0,0.0,0.0,0,20180122
6952091,99977,2018-01-22,MA2,NaN,22234.0,13.0,0.0,NaN,8.0,0.0,0.0,NaN,28.0,NaN,NaN,0,20180122


In [52]:

separator = '<SEP>'  # 使用的分隔符
total_data = []
for failure in tqdm(failure_data.itertuples()):
    # 如果当前时间 - lookback_days > start_date, 则计算超过的时间
    if failure.failure_time > end_date + timedelta(days=1):
        days = (failure.failure_time - end_date).days
        print(f'Failure time: {failure.failure_time}, days: {days}')
        
        # 删除 combined_data 中的 start_date 和 start_date+1 数据
        range_csv = range_csv[~range_csv['date'].isin([start_date.strftime('%Y%m%d'), (start_date + timedelta(days=1)).strftime('%Y%m%d')])]
        
        # 加载并加入 end_date+1 和 end_date+2 的数据
        for i in range(1, 3):
            new_date = end_date + timedelta(days=i)
            new_date_str = new_date.strftime('%Y%m%d')
            file_name = f"{new_date_str}_processed.csv"
            
            try:
                new_data = pd.read_csv(os.path.join(smart_data_base_folder, file_name))
                new_data['date'] = new_date_str
                range_csv = pd.concat([range_csv, new_data], ignore_index=True)
            except FileNotFoundError:
                print(f"文件 {file_name} 未找到，跳过该文件")

        # 更新 start_date 和 end_date
        start_date = start_date + timedelta(days=2)
        end_date = end_date + timedelta(days=2)

        # print("更新后的 Combined Data:")
        # print(range_csv)
        # print(f"新 start_date: {start_date}, 新 end_date: {end_date}")
        # break
    
    # 从 failure 中获取 disk_id 和 model
    disk_id = failure.disk_id
    model = failure.model

    # 在 range_csv 中筛选出 disk_id 和 model 与当前 failure 相同的数据
    matching_data = range_csv[(range_csv['disk_id'] == disk_id) & (range_csv['model'] == model)]

    # # 打印或处理匹配到的数据
    # if not matching_data.empty:
    #     print(f"Matching data for disk_id={disk_id}, model={model}:")
    #     print(matching_data)
    # else:
    #     print(f"No matching data found for disk_id={disk_id}, model={model}")
    
    for start_idx in range(0, len(matching_data) - window_size + 1, step_size):
        end_idx = start_idx + window_size
        window = matching_data.iloc[start_idx:end_idx]  # 当前窗口数据
        
        # 序列化每个窗口的数据
        serialized_data = []
        for _, row in window.iterrows():
            # 将单天数据转换为 [ds, r_1, r_9, ...] 格式
            day_data = [row['ds']] + row.filter(regex='^r_').tolist()
            serialized_data.extend(day_data)  # 将单天数据加入序列
            serialized_data.append(separator)  # 插入分隔符
        
        serialized_data = serialized_data[:-1]  # 移除最后一个多余的分隔符
        
        total_data.append(serialized_data)
    print(f'len of total_data: {len(total_data)}')
    # print(failure)
    # 获取其中某一列
    # print(failure.failure_time)
    # break

0it [00:00, ?it/s]

1it [00:00,  1.37it/s]

len of total_data: 3


2it [00:01,  1.54it/s]

len of total_data: 6


...

len of total_data: 34


18it [00:15,  1.24it/s]

len of total_data: 34
Failure time: 2018-01-27 03:01:11, days: 1


18it [00:20,  1.13s/it]


KeyboardInterrupt: 

In [54]:
window.iloc[-1].label

1

In [1]:
import os
import pandas as pd
from datetime import timedelta
from tqdm import tqdm

def load_processed_data_for_day(file_path):
    """
    加载处理后的SMART数据文件（每个日期的文件）
    """
    return pd.read_csv(file_path)

def smart_data_generator(smart_data_base_folder, batch_size=10):
    """
    生成器函数，分批读取CSV文件
    """
    files = [f for f in os.listdir(smart_data_base_folder) if f.endswith('_processed.csv')]
    
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i + batch_size]
        batch_data = []
        
        for filename in batch_files:
            file_path = os.path.join(smart_data_base_folder, filename)
            try:
                smart_data = pd.read_csv(file_path)
                smart_data['ds'] = pd.to_datetime(smart_data['ds'])
                batch_data.append(smart_data)
            except Exception as e:
                print(f"Error reading file {filename}: {str(e)}")
                continue
                
        if batch_data:
            yield pd.concat(batch_data, ignore_index=True)

def process_smart_data_for_disk(disk_id, model, start_time, end_time, smart_data_generator):
    """
    处理单个磁盘的数据
    """
    relevant_data = []
    
    for batch_data in smart_data_generator:
        disk_data = batch_data[(batch_data['disk_id'] == disk_id) & 
                              (batch_data['model'] == model) & 
                              (batch_data['ds'] >= start_time) & 
                              (batch_data['ds'] <= end_time)]
        if not disk_data.empty:
            relevant_data.append(disk_data)
    
    return pd.concat(relevant_data, ignore_index=True) if relevant_data else pd.DataFrame()

def generate_features_and_labels(failure_data, smart_data_base_folder, lookback_days, window_size, step_size):
    """
    生成模型训练数据，按给定的滑动窗口选择特征和标签
    """
    all_data = []
    failure_data = failure_data.sort_values(by='failure_time')
    
    # 创建一个字典来存储已处理过的磁盘数据
    processed_disks = {}
    
    for failure in tqdm(failure_data.itertuples(), desc="Processing failure events"):
        failure_time = failure.failure_time
        disk_id = failure.disk_id
        model = failure.model
        
        # 计算时间范围
        start_time = failure_time - timedelta(days=lookback_days)
        end_time = failure_time
        
        # 获取磁盘数据
        disk_key = f"{disk_id}_{model}_{start_time}_{end_time}"
        if disk_key not in processed_disks:
            relevant_smart_data = process_smart_data_for_disk(
                disk_id, 
                model, 
                start_time, 
                end_time, 
                smart_data_generator(smart_data_base_folder)
            )
            processed_disks[disk_key] = relevant_smart_data
        else:
            relevant_smart_data = processed_disks[disk_key]

        # 滑动窗口处理
        for i in range(0, lookback_days - window_size + 1, step_size):
            window_start_time = failure_time - timedelta(days=(lookback_days - i))
            window_end_time = window_start_time + timedelta(days=window_size)
            
            window_data_for_window = relevant_smart_data[
                (relevant_smart_data['ds'] >= window_start_time) & 
                (relevant_smart_data['ds'] < window_end_time)
            ]
            
            if not window_data_for_window.empty:
                values = window_data_for_window.drop(columns=['disk_id', 'label']).values.tolist()
                last_window_data = window_data_for_window.iloc[-1]
                label = last_window_data['label']
                all_data.append({'data': values, 'label': label})
                yield {'data': values, 'label': label}
        # 清理不再需要的处理过的磁盘数据
        if len(processed_disks) > 100:  # 保持缓存大小在可控范围内
            processed_disks.clear()
            
    # return all_data

# 使用方式保持不变


In [ ]:
smart_data_base_folder = '/mnt/raid5/sum/card/storage/StreamDFP/dataset/processed_smart_data'
failure_file_path = '/mnt/raid5/sum/card/storage/StreamDFP/dataset/ssd_failure_label.csv'

failure_data = pd.read_csv(failure_file_path)
failure_data['failure_time'] = pd.to_datetime(failure_data['failure_time'])

lookback_days = 20
window_size = 10
step_size = 5

dataset = generate_features_and_labels(failure_data, smart_data_base_folder, lookback_days, window_size, step_size)
print(next(dataset))

Processing failure events: 0it [00:00, ?it/s]

In [8]:
import os
import pandas as pd
df_data = pd.read_csv("/mnt/raid5/sum/card/storage/StreamDFP/dataset/processed_smart_data/20180108_processed.csv")

In [9]:
type(df_data)

pandas.core.frame.DataFrame

In [10]:
print('单个csv文件数据的规模，行数：%d, 列数：%d' % (df_data.shape[0], df_data.shape[1]))

单个csv文件数据的规模，行数：345542, 列数：16


In [4]:
df_data.head()

,disk_id,ds,model,r_1,r_9,r_12,r_171,r_173,r_174,r_183,r_187,r_188,r_194,r_195,r_198,label
0,1,2018-01-08,MB1,NaN,1015.0,16.0,NaN,NaN,NaN,0.0,0.0,NaN,26.0,0.0,NaN,0
1,10000,2018-01-08,MA2,NaN,27122.0,18.0,0.0,NaN,15.0,0.0,0.0,NaN,26.0,NaN,NaN,0
2,100034,2018-01-08,MC1,0.0,2226.0,20.0,0.0,12.0,16.0,0.0,0.0,19.0,21.0,0.0,0.0,0
3,100039,2018-01-08,MA2,NaN,14976.0,18.0,0.0,NaN,13.0,0.0,0.0,NaN,27.0,NaN,NaN,0
4,100047,2018-01-08,MA2,NaN,30425.0,13.0,0.0,NaN,8.0,0.0,0.0,NaN,27.0,NaN,NaN,0


In [5]:
df_data.describe()

,disk_id,r_1,r_9,r_12,r_171,r_173,r_174,r_183,r_187,r_188,r_194,r_195,r_198,label
count,345542.000000,1.556570e+05,345527.000000,335759.000000,257696.000000,155657.000000,257698.000000,308036.000000,335759.000000,1.458900e+05,299525.000000,2.239510e+05,145892.000000,345542.000000
mean,61223.214559,9.688713e+08,14332.037919,25.883768,3.064576,22.333207,22.127618,3.476555,1.420456,7.657148e+02,24.019170,4.315305e+07,0.020782,0.000324
std,51055.696532,1.783724e+09,11120.241488,159.135721,282.019041,51.051952,165.794014,377.822162,239.019362,2.576359e+05,4.758325,2.126036e+08,1.209994,0.018001
min,0.000000,0.000000e+00,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,7.000000,0.000000e+00,0.000000,0.000000
25%,21089.000000,0.000000e+00,4114.000000,15.000000,0.000000,1.000000,9.000000,0.000000,0.000000,2.000000e+00,22.000000,0.000000e+00,0.000000,0.000000
50%,42487.000000,0.000000e+00,12011.000000,19.000000,0.000000,5.000000,15.000000,0.000000,0.000000,1.600000e+01,25.000000,0.000000e+00,0.000000,0.000000
75%,94458.750000,6.600000e+01,22714.000000,26.000000,0.000000,19.000000,21.000000,0.000000,0.000000,5.200000e+01,27.000000,0.000000e+00,0.000000,0.000000
max,200207.000000,4.272667e+10,38756.000000,51208.000000,46424.000000,1458.000000,51205.000000,119867.000000,97861.000000,9.826090e+07,139.000000,4.259553e+09,286.000000,1.000000


In [6]:
df_data.isnull().sum()

disk_id         0
ds              0
model           0
r_1        189885
r_9            15
r_12         9783
r_171       87846
r_173      189885
r_174       87844
r_183       37506
r_187        9783
r_188      199652
r_194       46017
r_195      121591
r_198      199650
label           0
dtype: int64

In [7]:
valid = df_data[df_data['label'] == 0]
failed = df_data[df_data['label'] == 1]
print("valid hdds:",len(valid))
print("failed hdds:",len(failed))

valid hdds: 345430
failed hdds: 112


In [11]:
df_data = df_data.fillna(0) 

In [12]:
df_data.head()

,disk_id,ds,model,r_1,r_9,r_12,r_171,r_173,r_174,r_183,r_187,r_188,r_194,r_195,r_198,label
0,1,2018-01-08,MB1,0.0,1015.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0
1,10000,2018-01-08,MA2,0.0,27122.0,18.0,0.0,0.0,15.0,0.0,0.0,0.0,26.0,0.0,0.0,0
2,100034,2018-01-08,MC1,0.0,2226.0,20.0,0.0,12.0,16.0,0.0,0.0,19.0,21.0,0.0,0.0,0
3,100039,2018-01-08,MA2,0.0,14976.0,18.0,0.0,0.0,13.0,0.0,0.0,0.0,27.0,0.0,0.0,0
4,100047,2018-01-08,MA2,0.0,30425.0,13.0,0.0,0.0,8.0,0.0,0.0,0.0,27.0,0.0,0.0,0
